In [1]:
import numpy as np
import pickle
import pandas as pd
import torch 
from GRU import GRUmodel
from Utils import *

In [46]:
current_layer = 8
params_path = "./newBoosting/param{}.pt".format(current_layer)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

trajSet = TrajDatasetNoGraph("./traj_data_train.pkl", 8, hour_holiday="traj_hour_holiday_train.pkl")
rawroadfeat = RoadFeatures("./road_features_with_lengths.pkl", "../database/data/road.csv")

gru = GRUmodel(rawroadfeat.n_features, 128, device).to(device)
gru.load_state_dict(torch.load(params_path))

<All keys matched successfully>

In [4]:
# w = np.ones((len(trajSet)), dtype=np.float32)
# with open("./newBoosting/weight0.pkl", "wb") as f:
#     pickle.dump(w,f)

In [47]:
ret = predict_all_trajs(trajSet, gru, rawroadfeat)
ret

tensor([[37.5000, 19.4657],
        [ 5.5833,  5.1486],
        [23.8167, 20.6343],
        ...,
        [ 7.8500, 14.6502],
        [20.3833, 14.3791],
        [ 7.5667,  9.4011]], device='cuda:0')

In [48]:
threshold = 0.7
if current_layer != 0:
    with open("./newBoosting/alpha.pkl", "rb") as f:
        alpha = pickle.load(f)

with open("./newBoosting/weight{}.pkl".format(current_layer), "rb") as f:
    weight = pickle.load(f)

delta = ret[:,0] - ret[:,1]
rate = torch.abs(delta / ret[:,0])
# rate大于threshold的认为是错的，否则认为是对的.
err = np.float32((rate > threshold).detach().cpu().numpy())
print("超出阈值个数:{}".format(np.sum(err)))
e_m = float(np.sum(weight * err) / np.sum(weight))
alpha_m = 0.5 * np.log((1-e_m) / e_m)
print("e_m: {}, alpha_m: {}".format(e_m, alpha_m))
# 错的是-1，对的是1.
yH = -1 * err + (1.0 - err)
new_weight = np.exp(-alpha_m * yH) * weight
#new_weight /= np.sum(new_weight)
print(new_weight)
print(new_weight.min(), new_weight.max())

with open("./newBoosting/weight{}.pkl".format(current_layer+1), "wb") as f:
    pickle.dump(new_weight, f)
alpha_m = np.array([alpha_m], dtype=np.float32)
if current_layer != 0:
    alpha = np.concatenate((alpha, alpha_m))
else:
    alpha = alpha_m
print("new_alpha: {}".format(alpha))
with open("./newBoosting/alpha.pkl", "wb") as f:
    pickle.dump(alpha, f)

超出阈值个数:1620.0
e_m: 0.5286245942115784, alpha_m: -0.05731185586151965
[0.21208006 0.21208006 0.21208006 ... 1.2067821  0.414297   0.34888747]
0.18500046 5.405391
new_alpha: [ 0.27714676  0.02565434  0.03200861  0.5425863   0.51845974  0.07494448
  0.14829388 -0.01099094 -0.05731186]
